
# Prerequisite

In [ ]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset

In [ ]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

# Basic pipeline job

## Build pipeline

In [ ]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from pathlib import Path

parent_dir = '.'


def generate_dsl_pipeline() -> Pipeline:
    # 1. Load component funcs
    a_func = dsl.load_component(yaml_file=parent_dir + "/componentA.yml")
    b_func = dsl.load_component(yaml_file=parent_dir + "/componentB.yml")
    c_func = dsl.load_component(yaml_file=parent_dir + "/componentC.yml")

    # 2. Construct pipeline
    @dsl.pipeline(
        compute="cpu-cluster",
        description="Basic Pipeline Job with 3 Hello World components",
    )
    def sample_pipeline():
        componentA_job = a_func()
        componentB_job = b_func()
        componentC_job = c_func()

    pipeline = sample_pipeline()
    return pipeline

# Submit pipeline job

In [ ]:
# create pipeline instance
pipeline = generate_dsl_pipeline()
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="basic_pipeline", continue_run_on_step_failure=True)